In [1]:
from scm_irl.env.scm_irl_env import ScmIrlEnv
from sllib.conversions.geo_conversions import north_east_to_lat_lon, mps2knots, lat_lon_to_north_east
import numpy as np
import wandb
from wandb.integration.sb3 import WandbCallback


import gymnasium as gym
# Check if the environment is registered

gym.register(
    id='ScmIrl-v0',
    entry_point='scm_irl.env:ScmIrlEnv',
    max_episode_steps=500,
)


path = "/home/rolando/Documents/DTU/SCMarine/ais-and-charts-sample/ais_data/scenario_2a66ceaf61"


env = ScmIrlEnv(path, mmsi=215811000, awareness_zone = [200, 500, 200, 200], start_time_reference=1577905000.0)

env.reset()

#env.step(np.array([1.0,1.0]))

mmsi: 215811000, start_time: 1577905000.0, end_time: 1577906100.0


({'observation_matrix': array([[50., 50., 50., ..., 50., 50., 50.],
         [50., 50., 50., ..., 50., 50., 50.],
         [50., 50., 50., ..., 50., 50., 50.],
         ...,
         [50., 50., 50., ..., 50., 50., 50.],
         [50., 50., 50., ..., 50., 50., 50.],
         [50., 50., 50., ..., 50., 50., 50.]], dtype=float32),
  'agent_state': array([1.94430761e+03, 2.61866356e+03, 4.99006800e+00, 3.91923380e-01]),
  'target': array([6221.38466074, 6272.32500012])},
 {})

In [2]:
# import stablebaselines3

config = {
    "policy_type": "MultiInputPolicy",
    "total_timesteps": 100000,
    "env_name": "ScmIrl-v0",
}

run = wandb.init(
    project="scmirl",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)


from stable_baselines3 import PPO

model = PPO("MultiInputPolicy", env, verbose = 1)

model.learn(total_timesteps=100000, 
            callback=WandbCallback(
                gradient_save_freq=100,
                model_save_path=f"models/{run.id}",
                verbose=2,),
            log_interval=10,
            tb_log_name="ppo",
            reset_num_timesteps=False,
            progress_bar=True,
                )

model.save("ppo_ais")

run.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rolando-esq (dtu-projects). Use `wandb login --relogin` to force relogin


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 110       |
|    ep_rew_mean     | -3.34e+04 |
| time/              |           |
|    fps             | 19        |
|    iterations      | 1         |
|    time_elapsed    | 103       |
|    total_timesteps | 2048      |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 110          |
|    ep_rew_mean          | -3.33e+04    |
| time/                   |              |
|    fps                  | 16           |
|    iterations           | 2            |
|    time_elapsed         | 241          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0027384334 |
|    clip_fraction        | 0.00117      |
|    clip_range           | 0.2        